# What is Coding?

## Intro to Python and Google Colab

To run the code right here in this tab, click the rocket ship icon {fa}`rocket` and choose {fa}`caret-right` Live Code.

```{admonition} Alternatively, use Google Colab
:class: dropdown
You can run the following code using Google Colab, which runs on your browser (no installations required). Click the rocket ship icon {fa}`rocket` and choose Colab.

Note: When you run a block of code for the first time, Google Colab will say `Warning: This notebook was not authored by Google`. Please click `Run Anyway`.
```

Let's go through the basics below:

In [ ]:
print("Hello World!")

Hello World!


Python is a powerful tool for processing and analyzing data - perfect for this project. Let's explore some of the basic functionality.
## The Basics
A **variable** can be used to store values calculated in expressions and then used for other calculations.

Defining variables is straightforward, but in good naming practice use an underscore (_) rather than a space ( ).

In [ ]:
# Setting up two variables
variable_one = 1
variable_two = 2
# Performing a calculation using those variables
variable_three = variable_one + variable_two
# Use print OR name the variable to show the output
print(f"Your answer is {variable_three}.")

variable_three

Your answer is 3.


3

An **integer** is just a number (ex. 4). A **float** is a decimal number and a **string** is text. These types of values have specific rules that allow them to be displayed together.

If you look at the previous example you see
`print("Your answer is", variable_three,".")`

This is a combination of string and float values, with commas and quotation marks separating them within the parenthesis. This formatting allows output with no error.

In [ ]:
weather_forecast = "Hot"
type(weather_forecast)

str

In [ ]:
daily_temperature = 85.0
type(daily_temperature)

float

You can redefine variables or convert their types, to redefine, you take the original variable and assign it a different value.

In [ ]:
daily_temperature = "85.0"
type(daily_temperature)

str

In [ ]:
updated_temperature = float(daily_temperature)
# We can confirm the type has changed by checking the type of forecastHigh or by checking the output of a code cell with the variable.

type(updated_temperature)

float

So the value stays the same, but the variable type changes.

In [ ]:
print(updated_temperature)

85.0


**Functions** are blocks of code that you use for a specific task that are easy to reuse.                          
We'll define our first function in order to convert Fahrenheit to Celsius.


*  `def` is the keyword
*   `celsius_to_fahr` is the function name
* `(temp)` is the parameter





In [ ]:
def celsius_to_fahr(temp):
    return 9 / 5 * temp + 32

To **call** a function, it's the same as asking the terminal to `print`

In [ ]:
freezing_point = celsius_to_fahr(0)
print(f"The freezing point of water in Fahrenheit is: {freezing_point}")

The freezing point of water in Fahrenheit is: 32.0


## Intro to Google Earth Engine (GEE)

1. Go to [console.cloud.google.com](https://console.cloud.google.com/) and sign into your Google account if you're not already signed in. You'll see the screen below. Click "Create Project."
2. Enter a name like "emerge-data" and click "Create." This name is your project ID. Note: If you are using a .edu or an organizational email, this page may look different and you may have other options or limitations.
*Make note of your project ID!*
3. After you have created your cloud project, return to this code notebook.

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library. Make sure you change "emerge-data" to your project ID
ee.Initialize(project='emerge-data')

### Basic Terminology - Using Collections in GEE

**Features** are geometric objects with a list of properties
> ee.Feature

**Images** are similar to features, but can have several bands


> ee.Image


**Collections** are groups of features or images


> ee.FeatureCollection or ee.ImageCollection





When you go into the details of an individual dataset, you can find the snippet of how to import that specific dataset

For this example, we’ll be using the MODIS Terra Land Surface Temperature (LST) dataset as well as the USGS Ground Elevation (ELV)

We'll import those two datasets now


In [ ]:
# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/061/MOD11A1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

Datasets include several sets of information across several bands, with some datasets containing daily imagery in a 1km resolution (like the MODIS LST dataset we’re using) to one image a year in a 30m resolution for others.

The LST Collection uses these bands


*   LST_Day_1km: Daytime Land Surface Temperature
*   Day_view_time: Local time of day observation
* LST_Night_1km: Nighttime Land Surface Temperature

With so many images within the collection, we need to filter those images using dates

Use either of these for filtering
```
filterDate()
select()
```


In [ ]:
# Initial date of interest (inclusive).
i_date = '2017-01-01'

# Final date of interest (exclusive).
f_date = '2020-01-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

Next, we’ll define two points of interest (POIs) in relation to what we want to look at. Since we’re looking at Land Surface Emissivity (land temp), we’ll make a comparison of the temperature in an urban area versus a rural area.

In [ ]:
# Define the urban location of interest as a point near Miami, Florida.
urban_lon = -80.196432
urban_lat = 25.779766
urban_poi = ee.Geometry.Point(urban_lon, urban_lat)

# Define the rural location of interest as a point away from the city in Homestead, Florida.
rural_lon = -80.4998113
rural_lat = 25.3933527
rural_poi = ee.Geometry.Point(rural_lon, rural_lat)

The MODIS emissivity dataset that we’re using should be corrected by a scale of 0.02 to get units of Kelvin. You can find this in the catalog of the dataset that you’re using and going into the bands.

*Remember to convert your values for a final answer!*

We need the average land temperature of the two POIs we’re looking at, so we’ll use mean() .


In [ ]:
scale = 1000  # scale in meters

# Function to convert LST to Celsius and get mean LST.
def get_mean_lst(poi, scale):
    mean_lst = lst.mean().sample(poi, scale).first().get('LST_Day_1km').getInfo()
    return round(mean_lst * 0.02 - 273.15, 2)  # Convert to Celsius

# Get mean LST for both urban and rural points.
urban_lst = get_mean_lst(urban_poi, scale)
rural_lst = get_mean_lst(rural_poi, scale)

print ("The mean land surface temperature for the urban point is",urban_lst,"°C")
print ("The rural  land surface temperature for the urban point is",rural_lst,"°C")

# Maybe add an example for students to use their own coordinates/database?
# Maybe show getregion to export to GeoTIFF with pandas??
# https://developers.google.com/earth-engine/api_docs

The mean land surface temperature for the urban point is 32.36 °C
The rural  land surface temperature for the urban point is 26.56 °C
